In [44]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
%matplotlib widget
import matplotlib.pyplot as plt
tf.autograph.set_verbosity(0)
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)


In [6]:
def my_softmax(z):
    r = len(z)
    a = []
    for i in range(r):
        q = np.exp(z[i]) / sum(np.exp(z))
        a.append(q)    
    return a

In [7]:
z = np.array([1., 2.,3.,4.])
#softmax from our function
a = my_softmax(z)
#tensorflow softmax
atf = tf.nn.softmax(z)
print('my_softmax: ',a)
print('tensorflow: ',atf)


my_softmax:  [0.03205860328008499, 0.08714431874203257, 0.23688281808991013, 0.6439142598879722]
tensorflow:  tf.Tensor([0.0320586  0.08714432 0.23688282 0.64391426], shape=(4,), dtype=float64)


## DataSet

In [9]:
def load_dataset():
    x_train = np.load('data/X.npy')
    y_train = np.load('data/y.npy')
    return x_train,y_train

In [10]:
x_train, y_train = load_dataset()

In [12]:
print ('The first element of X is: ', x_train[0])

The first element of X is:  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+

In [14]:
print ('The first element of y is: ', y_train[0,0])
print ('The last element of y is: ', y_train[-1,0])

The first element of y is:  0
The last element of y is:  9


In [16]:
print ('The shape of X is: ' + str(x_train.shape))
print ('The shape of y is: ' + str(y_train.shape))

The shape of X is: (5000, 400)
The shape of y is: (5000, 1)


## Visualizing the Data

In [23]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

m, n = x_train.shape

fig, axes = plt.subplots(8,8, figsize=(5,5))
fig.tight_layout(pad=0.13,rect=[0, 0.03, 1, 0.91]) #[left, bottom, right, top]

#fig.tight_layout(pad=0.5)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
for i,ax in enumerate(axes.flat):
    # Select random indices
    random_index = np.random.randint(m)
    
    # Select rows corresponding to the random indices and
    # reshape the image
    X_random_reshaped = x_train[random_index].reshape((20,20)).T
    
    # Display the image
    ax.imshow(X_random_reshaped, cmap='gray')
    
    # Display the label above the image
    ax.set_title(y_train[random_index,0])
    ax.set_axis_off()
    fig.suptitle("Label, image", fontsize=14)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Model representation

The parameters have dimensions that are sized for a neural network with  25  units in layer 1,  15  units in layer 2 and  10  output units in layer 3, one for each digit.

Recall that the dimensions of these parameters is determined as follows:

If network has  𝑠𝑖𝑛  units in a layer and  𝑠𝑜𝑢𝑡  units in the next layer, then
𝑊  will be of dimension  𝑠𝑖𝑛×𝑠𝑜𝑢𝑡 .
𝑏  will be a vector with  𝑠𝑜𝑢𝑡  elements
Therefore, the shapes of W, and b, are

layer1: The shape of W1 is (400, 25) and the shape of b1 is (25,)

layer2: The shape of W2 is (25, 15) and the shape of b2 is: (15,)

layer3: The shape of W3 is (15, 10) and the shape of b3 is: (10,)

In [30]:
tf.random.set_seed(1234) # for consistent results
model = Sequential([
    #input layer
    tf.keras.Input(shape=(400,)),
    #hidden layers
    Dense(25,activation = 'relu', name = 'L1'),
    Dense(15, activation = 'relu', name = 'L2'),
    #output layer
    Dense(10, activation = 'linear', name = 'L3')
], name = 'my_model')

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [31]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L1 (Dense)                  (None, 25)                10025     
                                                                 
 L2 (Dense)                  (None, 15)                390       
                                                                 
 L3 (Dense)                  (None, 10)                160       
                                                                 
Total params: 10,575
Trainable params: 10,575
Non-trainable params: 0
_________________________________________________________________


In [32]:
[layer1, layer2, layer3] = model.layers

In [33]:
#### Examine Weights shapes
W1,b1 = layer1.get_weights()
W2,b2 = layer2.get_weights()
W3,b3 = layer3.get_weights()
print(f"W1 shape = {W1.shape}, b1 shape = {b1.shape}")
print(f"W2 shape = {W2.shape}, b2 shape = {b2.shape}")
print(f"W3 shape = {W3.shape}, b3 shape = {b3.shape}")

W1 shape = (400, 25), b1 shape = (25,)
W2 shape = (25, 15), b2 shape = (15,)
W3 shape = (15, 10), b3 shape = (10,)


In [45]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)

history = model.fit(
    x_train,y_train,
    epochs=40
)

Epoch 1/40
157/157 [==============================] - 1s 2ms/step - loss: 0.0306
Epoch 2/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0281
Epoch 3/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0250
Epoch 4/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0226
Epoch 5/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0219
Epoch 6/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0192
Epoch 7/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0196
Epoch 8/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0174
Epoch 9/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0222
Epoch 10/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0158
Epoch 11/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0125
Epoch 12/40
157/157 [==============================] - 0s 2ms/step - loss: 0.0118
Epoch 13/40
157/157 [====

# Prediction

In [42]:
  """ display a single digit. The input is one digit (400,). """
def display_a_digit(X):
    fig, ax = plt.subplots(1,1, figsize=(0.5,0.5))
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False   
    X_reshaped = X.reshape((20,20)).T
    # Display the image
    ax.imshow(X_reshaped, cmap='gray')
    plt.show()

In [46]:
image_of_two = x_train[1015]
display_a_digit(image_of_two)

prediction = model.predict(image_of_two.reshape(1,400))  # prediction

print(f" predicting a Two: \n{prediction}")
print(f" Largest Prediction index: {np.argmax(prediction)}")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 predicting a Two: 
[[-12.922445   -3.9776735   2.7201824  -3.5822005 -22.05717   -15.555857
  -19.23509    -8.497258  -10.322394  -12.857491 ]]
 Largest Prediction index: 2
